In [134]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [38]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [46]:
mask = (y >= 1)
X_filtered = X[mask]
y_filtered = y[mask]

In [48]:
class CustomLogReg:
    def __init__(self, lr=0.01, epochs=1000):
        """Инициализация гиперпараметров"""
        self.lr = lr          # Скорость обучения
        self.epochs = epochs  # Количество эпох обучения
        self.w = None         # Веса модели
        self.b = None         # Смещение
    
    def sigmoid(self, z):
        """Сигмоидная функция активации"""
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        """
        Обучение модели методом градиентного спуска.
        Параметры:
        X: матица признаков
        y: вектор истинных меток
        """
        n_samples, n_features = X.shape
        
        # Инициализация случайных весов и смещения
        self.w = np.random.rand(n_features)
        self.b = 0
        
        # Процесс обучения
        for _ in range(self.epochs):
            # Линейная комбинация признаков и текущих весов
            linear_model = np.dot(X, self.w) + self.b
            
            # Применение сигмоидной функции
            predicted = self.sigmoid(linear_model)
            
            # Производные для обновления весов и смещения
            dw = (1/n_samples) * np.dot(X.T, (predicted - y))   
            db = (1/n_samples) * np.sum(predicted - y)
            
            # Обновление весов и смещения
            self.w -= self.lr * dw
            self.b -= self.lr * db
    
    def predict(self, X):
        """
        Предсказание классов на новых данных.
        Параметр:
        X: матрица признаков
        Возвращает:
        y_pred: предсказанные классы
        """
        linear_model = np.dot(X, self.w) + self.b
        probabilities = self.sigmoid(linear_model)
        y_pred = [1 if prob > 0.5 else 0 for prob in probabilities]
        return y_pred
    
    def score(self, X, y):
        """
        Оценка доли правильных предсказаний.
        Параметры:
        X: матрица признаков
        y: вектор истинных меток
        Возвращает:
        accuracy: доля правильных предсказаний
        """
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        return accuracy

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
logreg=CustomLogReg()

In [68]:
logreg.fit(X_train, y_train)

In [72]:
y_pred = logreg.predict(X_test)

In [80]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.3000


In [104]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [136]:
class LogisticRegressionRMSProp:
    def __init__(self, learning_rate=0.01, decay_factor=0.9, eps=1e-8, num_iterations=1000):
        self.learning_rate = learning_rate
        self.decay_factor = decay_factor
        self.eps = eps
        self.num_iterations = num_iterations
        self.theta = None
        self.v = None  # Накопленная сумма градиентов
        
    def fit(self, X, y):
        # Добавляем единицу в начало каждого примера для учета свободного члена (bias)
        X = np.insert(X, 0, 1, axis=1)
        
        # Инициализация весов случайными значениями
        self.theta = np.random.rand(X.shape[1])
        self.v = np.zeros_like(self.theta)
        
        # RMSProp
        for _ in range(self.num_iterations):
            # Рассчитываем линейную комбинацию
            z = np.dot(X, self.theta)
            
            # Пропускаем через сигмоиду
            h = sigmoid(z)
            
            # Вычисляем градиент функции потерь
            gradient = np.dot(X.T, (h - y)) / len(y)
            
            # Обновляем v — накапливаем средние квадратичные отклонения градиентов
            self.v = self.decay_factor * self.v + (1 - self.decay_factor) * (gradient ** 2)
            
            # Обновляем веса с коррекцией по RMSProp
            self.theta -= self.learning_rate * gradient / (np.sqrt(self.v) + self.eps)
    
    def predict(self, X):
        # Добавляем единицу в начало каждого примера
        X = np.insert(X, 0, 1, axis=1)
        
        # Рассчитываем вероятность принадлежности к классу 1
        probabilities = sigmoid(np.dot(X, self.theta))
        
        # Преобразуем вероятности в бинарные предсказания
        return np.where(probabilities >= 0.5, 1, 0)

In [90]:
logregRMS=LogisticRegressionRMSProp()

In [106]:
logregRMS.fit(X_train, y_train)

In [110]:
y_pred = logregRMS.predict(X_test)

In [116]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy using RMSProp: {accuracy:.4f}')

Test Accuracy using RMSProp: 0.3000


In [120]:
class LogisticRegressionNAdam:
    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, eps=1e-8, num_iterations=1000):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.num_iterations = num_iterations
        self.theta = None
        self.m = None  # Первое среднее (Momentum)
        self.v = None  # Второе среднее (Variance)
    
    def fit(self, X, y):
        # Добавляем единицу в начало каждого примера для учета свободного члена (bias)
        X = np.insert(X, 0, 1, axis=1)
        
        # Инициализация весов случайными значениями
        self.theta = np.random.rand(X.shape[1])
        self.m = np.zeros_like(self.theta)
        self.v = np.zeros_like(self.theta)
        
        # NAdam
        for t in range(1, self.num_iterations + 1):
            # Рассчитываем линейную комбинацию
            z = np.dot(X, self.theta)
            
            # Пропускаем через сигмоиду
            h = sigmoid(z)
            
            # Вычисляем градиент функции потерь
            gradient = np.dot(X.T, (h - y)) / len(y)
            
            # Обновляем m и v
            self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
            self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)
            
            # Исправляем смещенность
            m_corr = self.m / (1 - self.beta1 ** t)
            v_corr = self.v / (1 - self.beta2 ** t)
            
            # Обновляем веса с поправкой на Nesterov momentum
            self.theta -= self.learning_rate * ((self.beta1 * m_corr) + (1 - self.beta1) * gradient) / (np.sqrt(v_corr) + self.eps)
    
    def predict(self, X):
        # Добавляем единицу в начало каждого примера
        X = np.insert(X, 0, 1, axis=1)
        
        # Рассчитываем вероятность принадлежности к классу 1
        probabilities = sigmoid(np.dot(X, self.theta))
        
        # Преобразуем вероятности в бинарные предсказания
        return np.where(probabilities >= 0.5, 1, 0)

In [124]:
logreg_nadam = LogisticRegressionNAdam()

In [126]:
logreg_nadam.fit(X_train, y_train)

In [128]:
y_pred = logreg_nadam.predict(X_test)

In [132]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy using NAdam: {accuracy:.4f}')

Test Accuracy using NAdam: 0.3000


In [158]:
methods = [
        {"method": "GD", "cls": CustomLogReg},
        {"method": "RMSProp", "cls": LogisticRegressionRMSProp},
        {"method": "NAdam", "cls": LogisticRegressionNAdam}
    ]
    
results = []  # список для хранения результатов
    
for method_dict in methods:
    start_time = time.time()  # старт таймера
        
        # создаем экземпляр модели и выполняем обучение
    clf = method_dict["cls"]()
    clf.fit(X_train, y_train)
        
        # измеряем время окончания
    end_time = time.time()
    elapsed_time = end_time - start_time
        
        # делаем прогнозы и считаем точность
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
        
        # добавляем результаты в таблицу
    results.append({"method": method_dict["method"], "metric": accuracy, "time": elapsed_time})
    
    # Выводим таблицу
print("Результаты сравнения методов оптимизации:")
print("| Метод      | Метрика   | Время работы (секунд) |")
print("|------------|-----------|-----------------------|")
for result in results:
    print(f"| {result['method']}      | {result['metric']:.4f} |      {result['time']:.4f} |")

Результаты сравнения методов оптимизации:
| Метод      | Метрика   | Время работы (секунд) |
|------------|-----------|-----------------------|
| GD      | 0.3000 |      0.0800 |
| RMSProp      | 0.3000 |      0.1199 |
| NAdam      | 0.3000 |      0.1379 |


GD (Gradien Descent): самый быстрый метод с временем около 0.08 секунд.
RMSProp: незначительно замедляется по сравнению с GD, демонстрируя время порядка 0.12 секунды.
NAdam: занимает наибольшее время среди всех методов — примерно 0.14 секунды.
точность одинаковая 0,30